In [21]:
import numpy as np
import pandas as pd
import os
import glob
import sys
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import colors
import cartopy.crs as ccrs  # Import cartopy ccrs
import cartopy.feature as cfeature  # Import cartopy common features
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

sys.path.insert(0, "/home/chalifour/code/master")
from fct_script.func_py import get_proj_extent
from matplotlib.patches import Patch
import fct_script.rpn_funcs_chris as rpn_chris
from tqdm.notebook import trange, tqdm
try:
    import rpnpy.librmn.all as rmn  # Module to read RPN files
    from rotated_lat_lon import RotatedLatLon  # Module to project field on native grid (created by Sasha Huziy)
except ImportError as err:
    print(f"RPNPY can only be use on the server. It can't be use on a personal computer."
          f"\nError throw :{err}")

In [22]:
# Date formating

date = '2022-03-25'
datetimeobject = datetime.strptime(date, '%Y-%m-%d')
new_format = datetimeobject.strftime('%Y%m')
#
run = pd.Timestamp(fr'{date} 00')

f_hr = 48
valid_time = run + pd.Timedelta(str(f_hr) + ' h')
timestamps = pd.date_range(run, valid_time, freq='1 H')
time = timestamps


# start = pd.date_range(start='2021-12-6 00',end='2021-12-7 19',freq='2H')
# end = pd.date_range(start='2021-12-6 02',end='2021-12-9 20',freq='2H')

In [23]:
# path formating

data_path = sorted(glob.glob(fr"/upslope/chalifour/projet_maitrise/"))[0]

graph_savepath = os.path.join(data_path, r'fig/graph_UL/data_visualisation')
HQ_preformat_savepath = os.path.join(data_path, r"data_parcivel/station_hq/station_gmon/Preformated_DB")
HQ_compiled_savepath = os.path.join(data_path, r"data_format-master/Data.nosync/station_gmon/Full_datasets")

DB_file_2022 = glob.glob(HQ_preformat_savepath + '/*.csv')

file_15min = glob.glob(HQ_compiled_savepath + '/*_15min.csv')

file_1h = glob.glob(HQ_compiled_savepath + '/dataset_1h.csv')


# uqam

file_uqam_1h = glob.glob(os.path.join(data_path + 'data_format-master/Data.nosync/site_uqam/Full_dataset/dataset_1h.csv'))

# momo
file_momo_1h = glob.glob(os.path.join(data_path + 'data_format-master/Data.nosync/site_neige/Full_dataset/dataset_1h.csv'))


In [24]:

dataframe_1h = pd.read_csv(file_1h[0], parse_dates=['date'])
dataframe_1h.set_index('date', inplace=True)

dataframe_uqam = pd.read_csv(file_uqam_1h[0], parse_dates=['date'])
dataframe_uqam.set_index('date', inplace=True)

dataframe_momo = pd.read_csv(file_momo_1h[0], parse_dates=['date'])
dataframe_momo.set_index('date', inplace=True)

In [25]:
# Load localisation station
lonlat_path = '/upslope/chalifour/projet_maitrise/data_format-master/Data.nosync/Disdrometres_coordonnées.csv'
df_disdro = pd.read_csv(lonlat_path, header=0)
df_disdro.set_index('Name', inplace=True)

In [26]:
phase_list = ['f_0','f_60','f_67','f_69','f_70']
phase_list_simulation =['RN','FR','MX','SN']
phase_list_simulation_pie =['RN','FR','MX_COLD','MX_HOT','SOL']
phase_str = ['None',  'Liquid', 'Freezing rain', 'Mix of Liquid and Solid with $T<0$ \u2103','Mix of Liquid and Solid with $T\geq0$ \u2103','Solid']

pType_color = ['k', 'tab:green', 'tab:red','tab:purple','tab:orange' ,'tab:blue',]  # ['firebrick','lime',    'm',       'orange']#

In [27]:
m, lonE2p5, latE2p5 = get_proj_extent()

{}
263.0 47.5 353.0 0.0


In [8]:
# load sim dataset station
begin,end = '2020-10','2021-07'
# 11 km
file_11km_stat = glob.glob(os.path.join(data_path + f'data_sim_station/dataset_stat_sim_11km_{begin.strip("-")}_{end.strip("-")}.csv'))

dataframe_11km_stat = pd.read_csv(file_11km_stat[0], parse_dates=['time'])
dataframe_11km_stat.set_index('time', inplace=True)

# uqam et momo 11 km

file_11km_Umomo = glob.glob(os.path.join(data_path + f'data_sim_station/dataset_UQAM_MOMO_sim_11km_{begin.strip("-")}_{end.strip("-")}.csv'))
dataframe_11km_Umomo = pd.read_csv(file_11km_Umomo[0], parse_dates=['time'])
dataframe_11km_Umomo.set_index('time', inplace=True)

# 2p5 km
file_2p5km_stat = glob.glob(os.path.join(data_path + f'data_sim_station/dataset_stat_sim_2p5km_{begin.strip("-")}_{end.strip("-")}.csv'))
dataframe_2p5km_stat = pd.read_csv(file_2p5km_stat[0], parse_dates=['time'])
dataframe_2p5km_stat.set_index('time', inplace=True)


# uqam et momo 2p5 km

file_2p5km_Umomo = glob.glob(os.path.join(data_path + f'data_sim_station/dataset_UQAM_MOMO_sim_2p5km_{begin.strip("-")}_{end.strip("-")}.csv'))
dataframe_2p5km_Umomo = pd.read_csv(file_2p5km_Umomo[0], parse_dates=['time'])
dataframe_2p5km_Umomo.set_index('time', inplace=True)

In [ ]:
# creat mx and sol categorie stat

list_df_11km=[]
for stat, subdf in dataframe_11km_stat.groupby('filename'):
    div = subdf[['RN','SN','PE','FR']]/subdf[['RN','SN','PE','FR']]
    sum = div.sum(axis=1)

    mask_sum = sum>1

    mask_qty = subdf[['RN','SN','PE','FR']]> 0.05
    mask_qty = mask_qty.sum(axis=1) > 1


    # On utilise les température pour déterminer les mix cold hot
    mask_temp_chaud = subdf['TT']>= 0.0
    mask_temp_froid = subdf['TT']< 0.0




    subdf.loc[mask_sum & mask_qty,'MX_COLD'] = subdf.loc[mask_sum & mask_qty & mask_temp_froid  ,['RN','SN','PE','FR']].sum(axis=1)
    subdf.loc[mask_sum & mask_qty,'MX_HOT'] = subdf.loc[mask_sum & mask_qty & mask_temp_chaud  ,['RN','SN','PE','FR']].sum(axis=1)
    subdf.loc[mask_sum & mask_qty & mask_temp_froid ,['RN','SN','PE','FR']] = 0
    subdf.loc[mask_sum & mask_qty & mask_temp_chaud ,['RN','SN','PE','FR']] = 0
    subdf['SOL'] = subdf[['SN','PE']].sum(axis=1)
    subdf.loc[np.isnan(subdf['MX_COLD']),'MX_COLD']=0
    subdf.loc[np.isnan(subdf['MX_HOT']),'MX_HOT']=0
    subdf.loc[np.isnan(subdf['SOL']),'SOL']=0
    list_df_11km.append(subdf)
    # dataframe_2p5km_stat.loc[dataframe_2p5km_stat['filename']==stat] = subdf

dataframe_11km_stat = pd.concat(list_df_11km)


list_df_2p5=[]
for stat, subdf in dataframe_2p5km_stat.groupby('filename'):
    div = subdf[['RN','SN','PE','FR']]/subdf[['RN','SN','PE','FR']]
    sum = div.sum(axis=1)

    mask_sum = sum>1

    mask_qty = subdf[['RN','SN','PE','FR']]> 0.05
    mask_qty = mask_qty.sum(axis=1) > 1


    # On utilise les température pour déterminer les mix cold hot
    mask_temp_chaud = subdf['TT']>= 0.0
    mask_temp_froid = subdf['TT']< 0.0




    subdf.loc[mask_sum & mask_qty,'MX_COLD'] = subdf.loc[mask_sum & mask_qty & mask_temp_froid  ,['RN','SN','PE','FR']].sum(axis=1)
    subdf.loc[mask_sum & mask_qty,'MX_HOT'] = subdf.loc[mask_sum & mask_qty & mask_temp_chaud  ,['RN','SN','PE','FR']].sum(axis=1)
    subdf.loc[mask_sum & mask_qty & mask_temp_froid ,['RN','SN','PE','FR']] = 0
    subdf.loc[mask_sum & mask_qty & mask_temp_chaud ,['RN','SN','PE','FR']] = 0
    subdf['SOL'] = subdf[['SN','PE']].sum(axis=1)
    subdf.loc[np.isnan(subdf['MX_COLD']),'MX_COLD']=0
    subdf.loc[np.isnan(subdf['MX_HOT']),'MX_HOT']=0
    subdf.loc[np.isnan(subdf['SOL']),'SOL']=0
    list_df_2p5.append(subdf)
    # dataframe_2p5km_stat.loc[dataframe_2p5km_stat['filename']==stat] = subdf

dataframe_2p5km_stat = pd.concat(list_df_2p5)
# uqam et momo
list_df_11km=[]
for stat, subdf in dataframe_11km_Umomo.groupby('filename'):

    div = subdf[['RN','SN','PE','FR']]/subdf[['RN','SN','PE','FR']]
    sum = div.sum(axis=1)

    mask_sum = sum>1

    mask_qty = subdf[['RN','SN','PE','FR']]> 0.05
    mask_qty = mask_qty.sum(axis=1) > 1


    # On utilise les température pour déterminer les mix cold hot
    mask_temp_chaud = subdf['TT']>= 0.0
    mask_temp_froid = subdf['TT']< 0.0




    subdf.loc[mask_sum & mask_qty,'MX_COLD'] = subdf.loc[mask_sum & mask_qty & mask_temp_froid  ,['RN','SN','PE','FR']].sum(axis=1)
    subdf.loc[mask_sum & mask_qty,'MX_HOT'] = subdf.loc[mask_sum & mask_qty & mask_temp_chaud  ,['RN','SN','PE','FR']].sum(axis=1)
    subdf.loc[mask_sum & mask_qty & mask_temp_froid ,['RN','SN','PE','FR']] = 0
    subdf.loc[mask_sum & mask_qty & mask_temp_chaud ,['RN','SN','PE','FR']] = 0
    subdf['SOL'] = subdf[['SN','PE']].sum(axis=1)
    subdf.loc[np.isnan(subdf['MX_COLD']),'MX_COLD']=0
    subdf.loc[np.isnan(subdf['MX_HOT']),'MX_HOT']=0
    subdf.loc[np.isnan(subdf['SOL']),'SOL']=0
    list_df_11km.append(subdf)
    # dataframe_2p5km_stat.loc[dataframe_2p5km_stat['filename']==stat] = subdf

dataframe_11km_Umomo = pd.concat(list_df_11km)


list_df_2p5=[]
for stat, subdf in dataframe_2p5km_Umomo.groupby('filename'):
    div = subdf[['RN','SN','PE','FR']]/subdf[['RN','SN','PE','FR']]
    sum = div.sum(axis=1)

    mask_sum = sum>1

    mask_qty = subdf[['RN','SN','PE','FR']]> 0.05
    mask_qty = mask_qty.sum(axis=1) > 1


    # On utilise les température pour déterminer les mix cold hot
    mask_temp_chaud = subdf['TT']>= 0.0
    mask_temp_froid = subdf['TT']< 0.0

    subdf.loc[mask_sum & mask_qty,'MX_COLD'] = subdf.loc[mask_sum & mask_qty & mask_temp_froid  ,['RN','SN','PE','FR']].sum(axis=1)
    subdf.loc[mask_sum & mask_qty,'MX_HOT'] = subdf.loc[mask_sum & mask_qty & mask_temp_chaud  ,['RN','SN','PE','FR']].sum(axis=1)
    subdf.loc[mask_sum & mask_qty & mask_temp_froid ,['RN','SN','PE','FR']] = 0
    subdf.loc[mask_sum & mask_qty & mask_temp_chaud ,['RN','SN','PE','FR']] = 0
    subdf['SOL'] = subdf[['SN','PE']].sum(axis=1)
    subdf.loc[np.isnan(subdf['MX_COLD']),'MX_COLD']=0
    subdf.loc[np.isnan(subdf['MX_HOT']),'MX_HOT']=0
    subdf.loc[np.isnan(subdf['SOL']),'SOL']=0

    list_df_2p5.append(subdf)
    # dataframe_2p5km_stat.loc[dataframe_2p5km_stat['filename']==stat] = subdf

dataframe_2p5km_Umomo = pd.concat(list_df_2p5)



In [ ]:
# Pie chart for each available station on the map
image_output_dpi = 200
fig = plt.figure(facecolor='white', figsize=(8 * 1.5, 3 *3.5))

spec = fig.add_gridspec(ncols=3, nrows=1,)
ax0 = fig.add_subplot(spec[0,0], projection=ccrs.LambertConformal())
ax1 = fig.add_subplot(spec[0, 1], projection=ccrs.LambertConformal())
ax2 = fig.add_subplot(spec[0, 2], projection=ccrs.LambertConformal())

list_axs=[ax0,ax1,ax2]

xll, yll = m.transform_point(lonE2p5[0, 0], latE2p5[0, 0], ccrs.PlateCarree())
xur, yur = m.transform_point(lonE2p5[-1, -1], latE2p5[-1, -1], ccrs.PlateCarree())

xm, ym = m.transform_point(-73.6053330201976, 45.52028815987258, ccrs.PlateCarree())

for ax_map in list_axs:
    # Set geographic features
    ax_map.add_feature(cfeature.OCEAN.with_scale('50m'), alpha=0.5)  # couche ocean
    ax_map.add_feature(cfeature.LAND.with_scale('50m'), facecolor='none')  # couche land
    # ax.add_feature(cfeature.LAKES.with_scale('50m'))      # couche lac
    ax_map.add_feature(cfeature.BORDERS.with_scale('50m'))  # couche frontieres
    # ax.add_feature(cfeature.RIVERS.with_scale('50m'))     # couche rivières
    coast = cfeature.NaturalEarthFeature(category='physical', scale='10m', facecolor='none',
                                         name='coastline')  # Couche côtières
    ax_map.add_feature(coast, edgecolor='black')
    ax_map.add_feature(cfeature.LAKES)
    states_provinces = cfeature.NaturalEarthFeature(category='cultural', name='admin_1_states_provinces_lines',
                                                    scale='10m', facecolor='none')  # Couche provinces
    ax_map.add_feature(states_provinces, edgecolor='grey')

# To help the layout of the figure after saving
list_stat_prob = ['GAREMANG','PIPMUA_G']

time_range_hours = pd.date_range(begin,end.replace("7","6"),freq='H',closed='right')[:-1]




for station, subdf in dataframe_1h.groupby('filename'):

    subdf = subdf.loc[begin:end.replace('7','5')]


    if len(subdf.index)!=0:
        if subdf['#nan_1h'].sum()/(4*len(time_range_hours))*100 < 10 and station not in list_stat_prob:

            # obs
            lon = df_disdro.loc[station].X
            lat = df_disdro.loc[station].Y

            list_count = []
            for i, num in enumerate(phase_list[1:]):

                if num == 'f_69':
                    mask_temp_froid = subdf['temp_moy']< 0.0
                    list_count.append(subdf.loc[mask_temp_froid,num].sum())

                    mask_temp_chaud = subdf['temp_moy']>= 0.0
                    list_count.append(subdf.loc[mask_temp_chaud,num].sum())

                else:
                    list_count.append(subdf[num].sum())
            list_count = np.array(list_count)


            if np.sum(list_count) != 0:
                lon_1, lat_1 = ccrs.LambertConformal().transform_point(lon, lat, ccrs.PlateCarree())

                ax_sub = inset_axes(ax0, width=0.4, height=0.4, loc=10,
                                    bbox_to_anchor=(lon_1, lat_1),
                                    bbox_transform=ax0.transData,
                                    borderpad=0)

                wedges, texts = ax_sub.pie(list_count, center=(lon, lat), colors=pType_color[1:], radius=0.7,
                                           wedgeprops={'linewidth': 0.5, "edgecolor": "k"})
                ax_sub.set_aspect("equal")




            else:
                list_stat_prob.append(station)
#  11 km
for station, subdf_11km in dataframe_11km_stat.groupby('filename'):
    # nb de nan plus grand que 10% du dataframe
    subdf_stat = dataframe_1h.loc[dataframe_1h['filename']==station]
    subdf_stat = subdf_stat.loc[begin:end.replace('7','5')]

    if len(subdf.index)!=0:
        if subdf_stat['#nan_1h'].sum()/(4*len(time_range_hours))*100 < 10 and station not in list_stat_prob:

            # obs
            lon = df_disdro.loc[station].X
            lat = df_disdro.loc[station].Y

            list_count = []
            for i, num in enumerate(phase_list_simulation_pie):
                subdf_11km = subdf_11km.loc[begin:end]
                list_count.append(subdf_11km[num].sum())
            list_count = np.array(list_count)

            if np.sum(list_count) != 0:
                lon_1, lat_1 = ccrs.LambertConformal().transform_point(lon, lat, ccrs.PlateCarree())

                ax_sub = inset_axes(ax1, width=0.4, height=0.4, loc=10,
                                    bbox_to_anchor=(lon_1, lat_1),
                                    bbox_transform=ax1.transData,
                                    borderpad=0)

                wedges, texts = ax_sub.pie(list_count, center=(lon, lat), colors=pType_color[1:], radius=0.7,
                                           wedgeprops={'linewidth': 0.5, "edgecolor": "k"})
                ax_sub.set_aspect("equal")



            else:
                pass

# 2p5 km

for station, subdf_2p5km in dataframe_2p5km_stat.groupby('filename'):
    # nb de nan plus grand que 10% du dataframe
    subdf_stat = dataframe_1h.loc[dataframe_1h['filename']==station]
    subdf_stat = subdf_stat.loc[begin:end.replace('7','5')]

    if len(subdf.index)!=0:
        if subdf_stat['#nan_1h'].sum()/(4*len(time_range_hours))*100 < 10 and station not in list_stat_prob:

            # obs
            lon = df_disdro.loc[station].X
            lat = df_disdro.loc[station].Y

            list_count = []
            for i, num in enumerate(phase_list_simulation_pie):
                subdf_2p5km = subdf_2p5km.loc[begin:end]
                list_count.append(subdf_2p5km[num].sum())
            list_count = np.array(list_count)

            if np.sum(list_count) != 0:
                lon_1, lat_1 = ccrs.LambertConformal().transform_point(lon, lat, ccrs.PlateCarree())

                ax_sub = inset_axes(ax2, width=0.4, height=0.4, loc=10,
                                    bbox_to_anchor=(lon_1, lat_1),
                                    bbox_transform=ax2.transData,
                                    borderpad=0)

                wedges, texts = ax_sub.pie(list_count, center=(lon, lat), colors=pType_color[1:], radius=0.7,
                                           wedgeprops={'linewidth': 0.5, "edgecolor": "k"})
                ax_sub.set_aspect("equal")



            else:
                pass

# uqam
name_uqam_momo = ['UQAM_PK','NEIGE']
lat_uqam_momo = [45.508594,47.322437368331876]
lon_uqam_momo = [-73.568741,-71.14730110000002]




list_count_uqam = []
for i, num in enumerate(phase_list[1:]):
    dataframe_uqam_ph = dataframe_uqam.loc[begin:end]
    if num == 'f_69':
        mask_temp_froid = dataframe_uqam_ph['temp_moy']< 0.0
        try:
            list_count_uqam.append(dataframe_uqam_ph.loc[mask_temp_froid,num].sum())
        except:
            list_count_uqam.append(0)

        mask_temp_chaud = dataframe_uqam_ph['temp_moy']>= 0.0

        try:
            list_count_uqam.append(dataframe_uqam_ph.loc[mask_temp_chaud,num].sum())
        except:
            list_count_uqam.append(0)

    else:
        try:
            list_count_uqam.append(dataframe_uqam_ph[num].sum())
        except:
            list_count_uqam.append(0)
list_count_uqam = np.array(list_count_uqam)

if np.sum(list_count_uqam) != 0:
    lon_uqam_1, lat_uqam_1 = ccrs.LambertConformal().transform_point(lon_uqam_momo[0], lat_uqam_momo[0], ccrs.PlateCarree())

    ax_sub = inset_axes(ax0, width=0.4, height=0.4, loc=10,
                        bbox_to_anchor=(lon_uqam_1, lat_uqam_1),
                        bbox_transform=ax0.transData,
                        borderpad=0)

    wedges, texts = ax_sub.pie(list_count_uqam, center=(lon_uqam_momo[0], lat_uqam_momo[0]), colors=pType_color[1:], radius=0.7,
                               wedgeprops={'linewidth': 0.5, "edgecolor": "k"})
    ax_sub.set_aspect("equal")






list_count_momo = []
for i, num in enumerate(phase_list[1:]):
    dataframe_momo_ph = dataframe_momo.loc[begin:end]
    if num == 'f_69':
        mask_temp_froid = dataframe_momo_ph['temp_moy']< 0.0
        try:
            list_count_momo.append(dataframe_momo_ph.loc[mask_temp_froid,num].sum())
        except:
            list_count_momo.append(0)

        mask_temp_chaud = dataframe_momo_ph['temp_moy']>= 0.0
        try:
            list_count_momo.append(dataframe_momo_ph.loc[mask_temp_chaud,num].sum())
        except:
            list_count_momo.append(0)

    else:
        try:
            list_count_momo.append(dataframe_momo_ph[num].sum())
        except:
            list_count_momo.append(0)


list_count_momo = np.array(list_count_momo)

if np.sum(list_count_momo) != 0:
    lon_momo_1, lat_momo_1 = ccrs.LambertConformal().transform_point(lon_uqam_momo[1], lat_uqam_momo[1], ccrs.PlateCarree())

    ax_sub = inset_axes(ax0, width=0.4, height=0.4, loc=10,
                        bbox_to_anchor=(lon_momo_1, lat_momo_1),
                        bbox_transform=ax0.transData,
                        borderpad=0)

    wedges, texts = ax_sub.pie(list_count_momo, center=(lon_uqam_momo[1], lat_uqam_momo[1]), colors=pType_color[1:], radius=0.7,
                               wedgeprops={'linewidth': 0.5, "edgecolor": "k"})
    ax_sub.set_aspect("equal")


# uqam et momo 11km
for idx_umomo_11km,stat_umomo in enumerate(name_uqam_momo) :
    subdf_11km_uqammomo = dataframe_11km_Umomo.loc[dataframe_11km_Umomo['filename'] == stat_umomo]

    list_count_11km = []
    for i, num in enumerate(phase_list_simulation_pie):

        list_count_11km.append(subdf_11km_uqammomo[num].sum())

    list_count_uqam = np.array(list_count_11km)

    if np.sum(list_count_11km) != 0:
        lon_uqam_1, lat_uqam_1 = ccrs.LambertConformal().transform_point(lon_uqam_momo[idx_umomo_11km], lat_uqam_momo[idx_umomo_11km], ccrs.PlateCarree())

        ax_sub = inset_axes(ax1, width=0.4, height=0.4, loc=10,
                            bbox_to_anchor=(lon_uqam_1, lat_uqam_1),
                            bbox_transform=ax1.transData,
                            borderpad=0)

        wedges, texts = ax_sub.pie(list_count_uqam, center=(lon_uqam_momo[idx_umomo_11km], lat_uqam_momo[idx_umomo_11km]), colors=pType_color[1:], radius=0.7,
                                   wedgeprops={'linewidth': 0.5, "edgecolor": "k"})
        ax_sub.set_aspect("equal")

# uqam et momo 2p5km
for idx_umomo_2p5km,stat_umomo in enumerate(name_uqam_momo) :
    subdf_2p5km_uqammomo = dataframe_2p5km_Umomo.loc[dataframe_2p5km_Umomo['filename'] == stat_umomo]
    list_count_2p5km = []
    for i, num in enumerate(phase_list_simulation_pie):

        list_count_2p5km.append(subdf_2p5km_uqammomo[num].sum())

    list_count_uqam = np.array(list_count_2p5km)

    if np.sum(list_count_uqam) != 0:
        lon_uqam_1, lat_uqam_1 = ccrs.LambertConformal().transform_point(lon_uqam_momo[idx_umomo_2p5km],
                                                                         lat_uqam_momo[idx_umomo_2p5km], ccrs.PlateCarree())

        ax_sub = inset_axes(ax2, width=0.4, height=0.4, loc=10,
                            bbox_to_anchor=(lon_uqam_1, lat_uqam_1),
                            bbox_transform=ax2.transData,
                            borderpad=0)

        wedges, texts = ax_sub.pie(list_count_uqam, center=(lon_uqam_momo[idx_umomo_2p5km], lat_uqam_momo[idx_umomo_2p5km]), colors=pType_color[1:], radius=0.7,
                                   wedgeprops={'linewidth': 0.5, "edgecolor": "k"})
        ax_sub.set_aspect("equal")

for ax_aff in list_axs:
    ax_aff.set_extent([xll + 17, xur - 9, yll + 4, yur - 1], crs=m)

time_text = f'{subdf.index[0].strftime("%Y-%m-%d")} to {subdf.index[-1].strftime("%Y-%m-%d")}'

ax2.annotate(f'{time_text}', xy=(0.15, 1.01), xycoords='axes fraction', fontsize=14)
# title
bbox = dict(boxstyle="square", fc="w")
ax0.annotate(f'(a) Observation', xy=(0.03, 0.93), bbox=bbox, xycoords='axes fraction', fontsize=12)
ax1.annotate(f'(b) CORDEX 0.11\u00b0', xy=(0.03, 0.93),bbox=bbox,xycoords='axes fraction', fontsize=12)
ax2.annotate(f'(c) Extended East 0.0225\u00b0', xy=(0.03, 0.93), bbox=bbox, xycoords='axes fraction', fontsize=12)
handles = [Patch(facecolor=color, label=label, edgecolor="k") for label, color in zip(phase_str[1:], pType_color[1:])]

plt.rcParams['legend.handlelength'] = 1

ax1.legend(handles=handles, loc='upper center', bbox_to_anchor=(0.5, -0.01),
          fancybox=True, shadow=True, ncol=2, fontsize=14)

plt.subplots_adjust(wspace=0.05)
image_savepath = fr"/upslope/chalifour/projet_maitrise/fig/carte_stat_pie/map_pie_{subdf.index[0].strftime('%Y')}_{subdf.index[-1].strftime('%Y')}.png"
fig.savefig(image_savepath, dpi=image_output_dpi, format='png', bbox_inches='tight')

plt.show()
plt.close()

In [17]:
# path simulation 11 km
# path_sim_11km_1 = glob.glob("/BIG1/roberge/Output/GEM5/Cascades_CORDEX/CLASS/Safe_versions/Spinup/NAM-11m_ERA5_GEM5_CLASS_NV_NA_newP3-SCPF_SN8_20yrs/Samples/NAM-11m_ERA5_GEM5_CLASS_NV_NA_newP3-SCPF_SN8_20yrs_20220*[0-5]")
# path_sim_11km_2 = glob.glob("/BIG1/roberge/Output/GEM5/Cascades_CORDEX/CLASS/Safe_versions/Spinup/NAM-11m_ERA5_GEM5_CLASS_NV_NA_newP3-SCPF_SN8_20yrs/Samples/NAM-11m_ERA5_GEM5_CLASS_NV_NA_newP3-SCPF_SN8_20yrs_20211*[0-2]")
# path_sim_11km = sorted(path_sim_11km_1+path_sim_11km_2)

# path sim 11km avec p3
path_sim_11km_1 = glob.glob("/chinook/roberge/Output/GEM5/Olivier/NAM-11m_ERA5_GEM50_PCPTYPEnil/Samples/NAM-11m_ERA5_GEM50_PCPTYPEnil_20210*[0-5]")

path_sim_11km_2 = glob.glob("/chinook/roberge/Output/GEM5/Olivier/NAM-11m_ERA5_GEM50_PCPTYPEnil/Samples/NAM-11m_ERA5_GEM50_PCPTYPEnil_20201*[0-2]")
path_sim_11km = sorted(path_sim_11km_1+path_sim_11km_2)


# path simulation 2.5 km
# path_sim_2p5_1 = glob.glob("/pampa/roberge/Output/GEM5/Cascades_CORDEX/CLASS/Safe_versions/Spinup/ECan_2.5km_NAM11mP3_newP3_CLASS_DEEPoff_SHALon/Samples/ECan_2.5km_NAM11mP3_newP3_CLASS_DEEPoff_SHALon_20210*[0-5]")
# path_sim_2p5_2 = glob.glob("/pampa/roberge/Output/GEM5/Cascades_CORDEX/CLASS/Safe_versions/Spinup/ECan_2.5km_NAM11mP3_newP3_CLASS_DEEPoff_SHALon/Samples/ECan_2.5km_NAM11mP3_newP3_CLASS_DEEPoff_SHALon_20201*[0-2]")
# path_sim_2p5km = sorted(path_sim_2p5_1+path_sim_2p5_2)

path_sim_2p5_1 = glob.glob("/chinook/roberge/Output/GEM5/Olivier/ECan_2.5km_NAM11mP3_GEM50_PCPTYPEnil/Samples/ECan_2.5km_NAM11mP3_GEM50_PCPTYPEnil_20210*[0-5]")
path_sim_2p5_2 = glob.glob("/chinook/roberge/Output/GEM5/Olivier/ECan_2.5km_NAM11mP3_GEM50_PCPTYPEnil/Samples/ECan_2.5km_NAM11mP3_GEM50_PCPTYPEnil_20201*[0-2]")
path_sim_2p5km = sorted(path_sim_2p5_1+path_sim_2p5_2)

In [18]:
# path_sim_2p5_2elev_dict_umomo = {'UQAM_PK': 70,
#                    'NEIGE': 664 }

# load simulation data Uqam et momo
# phase_list_simulation =['RN','SN','FR','PE']
# var_list_simulation =['UU','VV','TT','PR','RN','SN','FR','PE',]
var_list_simulation =['HR','HU']
phase_name_simulation  = ['Rain','Snow','Freezing\nRain','Refrozen\nprecipitation']

name_uqam_momo = ['UQAM_PK','NEIGE']
lat_uqam_momo = [45.508594,47.322437368331876]
lon_uqam_momo = [-73.568741,-71.14730110000002]

begin,end = '2020-10','2021-07'
timerange_month = pd.date_range(begin,end,freq='m',closed='right')

dict_idx_nearest_pt={'UQAM_PK':[],'NEIGE':[]}


# df_sim_stat = pd.DataFrame()
dict_gen = {'time':[],'filename':[]}
for phase in var_list_simulation:
    dict_gen[phase]=[]

for idx in range(len(timerange_month.month)-1):
    print(f'Getting {timerange_month[idx].year}-{timerange_month[idx].month} ')

    timerange_hour = pd.date_range(timerange_month[idx].strftime('%Y-%m'),timerange_month[idx+1].strftime('%Y-%m'),freq='1h')

    timerange_hour = timerange_hour[timerange_hour.month == timerange_month[idx].month]

    timerange_hour_datev = []

    for date_h in timerange_hour:
        timerange_hour_datev.append(rpn_chris.date_to_datev(date_h))
    try:
        rmn.fstcloseall(fid_2)
    except:
        pass


    # fid_2 = rmn.fstopenall(path_sim_11km[idx],rmn.FST_RO)
    fid_2 = rmn.fstopenall(path_sim_2p5km[idx],rmn.FST_RO)

    for j,phase in enumerate(var_list_simulation):
        print(f'getting {phase}')
        for hidx,datev in enumerate(timerange_hour_datev):

            # key1 = rmn.fstinf(fid, nomvar=phase)
            if phase in ['TT']:
                ip1_1_5m = rmn.ip1_val(1.5, rmn.LEVEL_KIND_MGL)
                rec = rmn.fstlir(fid_2, nomvar=phase, datev=datev,ip1=ip1_1_5m)
            elif phase in ['UU','VV']:
                ip1_10m = rmn.ip1_val(10, rmn.LEVEL_KIND_MGL)
                rec = rmn.fstlir(fid_2, nomvar=phase, datev=datev,ip1=ip1_10m)
            else:
                rec = rmn.fstlir(fid_2, nomvar=phase, datev=datev)


            for idx_stat in range(len(name_uqam_momo)):

                lon_stat_2 = lon_uqam_momo[idx_stat]
                lat_stat_2 = lat_uqam_momo[idx_stat]
                station = name_uqam_momo[idx_stat]

                if hidx ==0:
                    # dict_gen[phase].append(np.zeros((6,6)))
                    dict_gen[phase].append(0)
                    if j == 0:
                        dict_gen['time'].append(timerange_hour[hidx])
                        dict_gen['filename'].append(name_uqam_momo[idx_stat])
                else:

                    mygrid = rmn.readGrid(fid_2,rec)              # Get the grid information for the (LAM) Grid -- Reads the tictac's
                    latlondict = rmn.gdll(mygrid)               # Create 2-D lat and lon fields from the grid information
                    lat_var = latlondict['lat']                     # Assign 'lat' to 2-D latitude field
                    lon_var = latlondict['lon']

                    # Assign 'lon' to 2-D longitude field
                    # Close the RPN file

                    if len(dict_idx_nearest_pt[station]) == 0:
                        # path_elev_11km = "/BIG1/roberge/Output/GEM5/Cascades_CORDEX/CLASS/Safe_versions/Spinup/NAM-11m_ERA5_GEM5_CLASS_NV_NA_newP3-SCPF_SN8_20yrs/Samples/NAM-11m_ERA5_GEM5_CLASS_NV_NA_newP3-SCPF_SN8_20yrs_step0/dm2000010100_00000000p"
                        # path_elev_2p5km = "/pampa/roberge/Output/GEM5/Cascades_CORDEX/CLASS/Safe_versions/Spinup/ECan_2.5km_NAM11mP3_newP3_CLASS_DEEPoff_SHALon/Analysis/ECan_2.5km_NAM11mP3_newP3_CLASS_DEEPoff_SHALon_20150901-00000000"

                        # fid_elev = rmn.fstopenall(path_elev_11km,rmn.FST_RO)
                        # rec_elev = rmn.fstlir(fid_elev, nomvar='ME')
                        # mygrid_elev = rmn.readGrid(fid_elev,rec_elev)
                        # latlondict_elev = rmn.gdll(mygrid_elev)

                        # pyt = (lon_var-(360+lon_stat_2))**2+(lat_var-lat_stat_2)**2+(rec_elev['d']-elev_dict_umomo[name_uqam_momo[idx_stat]])**2
                        pyt = (lon_var-(360+lon_stat_2))**2+(lat_var-lat_stat_2)**2

                        idx_lat,idx_lon = np.unravel_index(np.nanargmin(pyt), pyt.shape)
                        dict_idx_nearest_pt[station] = [idx_lat,idx_lon]

                        var_pt = [rec['d'][idx_lat,idx_lon]]
                        # var_pt = [rec['d'][idx_lat-3:idx_lat+3,idx_lon-3:idx_lon+3]]
                        # lon_pt_array = lon_var[idx_lat-3:idx_lat+3,idx_lon-3:idx_lon+3]
                        # lat_pt_array = lat_var[idx_lat-3:idx_lat+3,idx_lon-3:idx_lon+3]
                        # np.save(f'//upslope/chalifour/projet_maitrise/data_sim_station/closest_point_array/lon_11km_{station}',lon_pt_array)
                        # np.save(f'//upslope/chalifour/projet_maitrise/data_sim_station/closest_point_array/lat_11km_{station}',lat_pt_array)

                    else:
                        idx_lat,idx_lon = dict_idx_nearest_pt[station]
                        var_pt=[rec['d'][idx_lat,idx_lon]]
                        # var_pt = [rec['d'][idx_lat-3:idx_lat+3,idx_lon-3:idx_lon+3]]

                    # var_pt = rmn.gdllsval(latlondict, [lat_stat_2], [lon_stat_2], rec['d']) #Get value of var interpolated to lat/lon point

                    if phase == 'TT':
                        dict_gen[phase].append(var_pt[0])
                    elif phase in ['UU','VV']:
                        dict_gen[phase].append(var_pt[0]*0.514444)
                    elif phase in ['PR','RN','SN','FR','PE']:
                        dict_gen[phase].append(var_pt[0]* 1000* 3600)
                    else:
                        dict_gen[phase].append(var_pt[0])

                    if j == 0:
                        dict_gen['time'].append(timerange_hour[hidx])
                        dict_gen['filename'].append(name_uqam_momo[idx_stat])
    rmn.fstcloseall(fid_2)

df_sim_stat = pd.DataFrame.from_dict(dict_gen)

df_sim_stat.set_index('time',inplace=True)
df_sim_stat.sort_values(['filename','time'],inplace=True)

# df_sim_stat.to_csv(f'//upslope/chalifour/projet_maitrise/data_sim_station/dataset_UQAM_MOMO_sim_11km_{begin.strip("_")}_{end.strip("_")}_HR.csv')
df_sim_stat.to_csv(f'//upslope/chalifour/projet_maitrise/data_sim_station/closest_point/dataset_UQAM_MOMO_sim_2p5kmP3_{begin.strip("_")}_{end.strip("_")}_HR.csv')


Getting 2020-10 
getting HR
getting HU
Getting 2020-11 
getting HR
getting HU
Getting 2020-12 
getting HR
getting HU
Getting 2021-1 
getting HR
getting HU
Getting 2021-2 
getting HR
getting HU
Getting 2021-3 
getting HR
getting HU
Getting 2021-4 
getting HR
getting HU
Getting 2021-5 
getting HR
getting HU


In [19]:
# elev_dict = {   'ARGENT': 641,
#                 'AUXLOUPS': 537.9,
#                 'BAUBERT': 541,
#                 'BETSIA_M': 403, # à valider
#                 'CABITUQG': 491,
#                 'LCABITUQ':491.0,
#                 'CONRAD': 433,
#                 'DIAMAND': 373,
#                 'GAREMANG': 778, # à valider
#                 'HARTJ_G': 460,
#                 'LACROI_G': 621,
#                 'LAFLAM_G': 519,
#                 'LAVAL': np.nan,
#                 'LBARDO_G': 486,
#                 'LEVASSEU': 466,
#                 'LOUISE_G': 397,
#                 'LOUIS': 315,
#                 'MOUCHA_M': 565,
#                 'NOIRS':385,
#                 'PARENT_G': 442,
#                 'PARLEUR': 485,
#                 'PERDRIX': 315,
#                 'PIPMUA_G': 566.2,
#                 'PORTO': 413,
#                 'ROMA_SE': 104,
#                 'ROUSSY_G': 456,
#                 'RTOULNUS': 688,
#                 'SM3CAM_G': 522,
#                 'STMARG_G': 461,
#                 'SAUTEREL':459 ,
#                 'WABISTAN': 565,
#                 'WEYMOU_G': 400,
#                 'NA':np.nan,
#                     }

In [28]:
# load simulation data

# phase_list_simulation =['RN','SN','FR','PE']
# var_list_simulation =['TT','PR','RN','SN','FR','PE']
var_list_simulation =['HR','HU']
phase_name_simulation  = ['Rain','Snow','Freezing\nRain','Refrozen\nprecipitation']
# for begin,end in zip(['2021-10'],['2022-07']):
for begin,end in zip(['2020-10','2021-10'],['2021-07','2022-07']):

    # path_sim_1 = glob.glob(f"/chinook/roberge/Output/GEM5/Olivier/NAM-11m_ERA5_GEM50_PCPTYPEnil/Samples/NAM-11m_ERA5_GEM50_PCPTYPEnil_{end[0:4]}0*[0-5]")
    # path_sim_2 = glob.glob(f"/chinook/roberge/Output/GEM5/Olivier/NAM-11m_ERA5_GEM50_PCPTYPEnil/Samples/NAM-11m_ERA5_GEM50_PCPTYPEnil_{begin[0:4]}1*[0-2]")

    path_sim_1 = glob.glob(f"/chinook/roberge/Output/GEM5/Olivier/ECan_2.5km_NAM11mP3_GEM50_PCPTYPEnil/Samples/ECan_2.5km_NAM11mP3_GEM50_PCPTYPEnil_{end[0:4]}0*[0-5]")
    path_sim_2 = glob.glob(f"/chinook/roberge/Output/GEM5/Olivier/ECan_2.5km_NAM11mP3_GEM50_PCPTYPEnil/Samples/ECan_2.5km_NAM11mP3_GEM50_PCPTYPEnil_{begin[0:4]}1*[0-2]")
    #
    path_sim_2p5km = sorted(path_sim_1+path_sim_2)

    # begin,end = '2021-10','2022-07'
    timerange_month = pd.date_range(begin,end,freq='m',closed='right')

    dict_idx_nearest_pt={}
    for station, subdf in dataframe_1h.groupby('filename'):
        subdf = subdf.loc[begin:end]
        if len(subdf.index)!=0:
            if subdf['#nan_1h'].sum()/(4*len(subdf.index))*100 < 10 or station is not ['GAREMANG','PIPMUA_G']:
                dict_idx_nearest_pt[station]=[]


    # df_sim_stat = pd.DataFrame()
    dict_gen = {'time':[],'filename':[]}
    for phase in var_list_simulation:
        dict_gen[phase]=[]
    timerange_hour_b = pd.date_range(timerange_month[0].strftime('%Y-%m'),timerange_month[len(timerange_month.month)-1].strftime('%Y-%m'),freq='1h')
    list_stat = [i for i,subdf in dataframe_1h.groupby('filename')]
    with tqdm(total=len(var_list_simulation) * len(timerange_hour_b) * len(list_stat)) as pbar:

        for idx in range(len(timerange_month.month)-1):
            # idx=timerange_month.month[-1]+1

            # print(f'Getting {timerange_month[idx].year}-{timerange_month[idx].month} ')

            timerange_hour = pd.date_range(timerange_month[idx].strftime('%Y-%m'),timerange_month[idx+1].strftime('%Y-%m'),freq='1h')

            timerange_hour = timerange_hour[timerange_hour.month == timerange_month[idx].month]

            timerange_hour_datev = []

            for date_h in timerange_hour:
                timerange_hour_datev.append(rpn_chris.date_to_datev(date_h))

            try:
                rmn.fstcloseall(fid)
            except:
                pass

            # fid = rmn.fstopenall(path_sim_11km[idx],rmn.FST_RO)
            fid = rmn.fstopenall(path_sim_2p5km[idx],rmn.FST_RO)
            for j,phase in enumerate(var_list_simulation):

                # print(f'getting {phase}')
                for hidx,datev in enumerate(timerange_hour_datev):
                    pbar.set_description(f'Processing {phase}: {timerange_month[idx].year}/{timerange_month[idx].month}/{timerange_hour[hidx].day} {timerange_hour[hidx].hour}:00 ')

                    # key1 = rmn.fstinf(fid, nomvar=phase)
                    if phase in ['TT']:
                        ip1_1_5m = rmn.ip1_val(1.5, rmn.LEVEL_KIND_MGL)
                        rec = rmn.fstlir(fid, nomvar=phase, datev=datev,ip1=ip1_1_5m)
                    elif phase in ['UU','VV']:
                        ip1_10m = rmn.ip1_val(10, rmn.LEVEL_KIND_MGL)
                        rec = rmn.fstlir(fid, nomvar=phase, datev=datev,ip1=ip1_10m)
                    else:
                        rec = rmn.fstlir(fid, nomvar=phase, datev=datev)
                    # rec = rmn.fstlir(fid, nomvar=phase, datev=datev)

                    for station, subdf in dataframe_1h.groupby('filename'):
                        subdf = subdf.loc[begin:end]

                        if len(subdf.index)!=0:
                            if subdf['#nan_1h'].sum()/(4*len(subdf.index))*100 < 10 or station is not ['GAREMANG','PIPMUA_G']:

                                lon_stat = df_disdro.loc[station].X
                                lat_stat = df_disdro.loc[station].Y

                                if hidx ==0:
                                    dict_gen[phase].append(0)
                                    if j == 0:
                                        dict_gen['time'].append(timerange_hour[hidx])
                                        dict_gen['filename'].append(station)
                                else:
                                    mygrid = rmn.readGrid(fid,rec)              # Get the grid information for the (LAM) Grid -- Reads the tictac's
                                    latlondict = rmn.gdll(mygrid)

                                    lat_var = latlondict['lat']                     # Assign 'lat' to 2-D latitude field
                                    lon_var = latlondict['lon']


                                    if len(dict_idx_nearest_pt[station]) == 0:

                                        pyt = (lon_var-(360+lon_stat))**2+(lat_var-lat_stat)**2
                                        idx_lat,idx_lon = np.unravel_index(np.nanargmin(pyt), pyt.shape)
                                        dict_idx_nearest_pt[station] = [idx_lat,idx_lon]

                                        var_pt = [rec['d'][idx_lat,idx_lon]]
                                    else:
                                        idx_lat,idx_lon = dict_idx_nearest_pt[station]

                                        var_pt = [rec['d'][idx_lat,idx_lon]]


                                   # Close the RPN file
                                   #  var_pt = rmn.gdllsval(latlondict, [lat_stat], [lon_stat], rec['d']) #Get value of var interpolated to lat/lon point

                                    if phase == 'TT':
                                        dict_gen[phase].append(var_pt[0])
                                    elif phase in ['UU','VV']:
                                        dict_gen[phase].append(var_pt[0]*0.514444)
                                    elif phase in ['PR','RN','SN','FR','PE']:
                                        dict_gen[phase].append(var_pt[0]* 1000* 3600)
                                    else:
                                        dict_gen[phase].append(var_pt[0])

                                    if j == 0:
                                        dict_gen['time'].append(timerange_hour[hidx])
                                        dict_gen['filename'].append(station)

                                pbar.update(1)

            rmn.fstcloseall(fid)



    df_sim_stat = pd.DataFrame.from_dict(dict_gen)

    df_sim_stat.set_index('time',inplace=True)
    df_sim_stat.sort_values(['filename','time'],inplace=True)

    df_sim_stat.to_csv(f'/upslope/chalifour/projet_maitrise/data_sim_station/closest_point/dataset_stat_sim_2p5kmP3_{begin.strip("_")}_{end.strip("_")}_HR.csv')
# df_sim_stat.to_csv(f'/upslope/chalifour/projet_maitrise/data_sim_station/closest_point_3d/dataset_stat_sim_11km_{begin.strip("_")}_{end.strip("_")}.csv')
# df_sim_stat.to_csv(f'/upslope/chalifour/projet_maitrise/data_sim_station/dataset_stat_sim_11km_{begin.strip("_")}_{end.strip("_")}.csv')

  0%|          | 0/361646 [00:00<?, ?it/s]

  0%|          | 0/361646 [00:00<?, ?it/s]